In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
from citipy import citipy
import numpy as np
from scipy.stats import linregress


In [2]:
num_cities = 500
lats = np.random.uniform(-90, 90, num_cities)
lngs = np.random.uniform(-180, 180, num_cities)
coordinates = list(zip(lats, lngs))

cities = []
for lat, lon in coordinates:
    city = citipy.nearest_city(lat, lon).city_name
    if city not in cities:
        cities.append((city, lat, lon))


In [ ]:
api_key = '850d6e457e84fc4c9f409b4b0ffd34f4'
url = "https://api.openweathermap.org/data/3.0/onecall"

cities_names = []
latitudes = []
longitudes = []
temperatures = []
humidities = []
cloudiness = []
wind_speeds = []

for city, lat, lon in cities:
    params = {
        'lat': lat,
        'lon': lon,
        'exclude': 'minutely,hourly,daily,alerts',
        'appid': api_key,
        'units': 'metric'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if response.status_code == 200:
        cities_names.append(city)
        latitudes.append(lat)
        longitudes.append(lon)
        temperatures.append(data['current']['temp'])
        humidities.append(data['current']['humidity'])
        cloudiness.append(data['current']['clouds'])
        wind_speeds.append(data['current']['wind_speed'])


In [ ]:
weather_data = pd.DataFrame({
    'City': cities_names,
    'Latitude': latitudes,
    'Longitude': longitudes,
    'Temperature': temperatures,
    'Humidity': humidities,
    'Cloudiness': cloudiness,
    'Wind Speed': wind_speeds
})


In [ ]:
weather_data['Hemisphere'] = np.where(weather_data['Latitude'] >= 0, 'Northern', 'Southern')


In [ ]:
def plot_regression(x, y, hemisphere, xlabel, ylabel):
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = f'y = {slope:.2f}x + {intercept:.2f}\nR-squared: {r_value**2:.2f}'
    
    plt.scatter(x, y, edgecolor='k', alpha=0.75)
    plt.plot(x, regress_values, color='red')
    
    plt.title(f'{hemisphere} Hemisphere: {ylabel} vs. {xlabel}')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.annotate(line_eq, (x.min(), y.max()*0.95), fontsize=12, color='red')
    plt.grid(True)


In [ ]:
northern_data = weather_data[weather_data['Hemisphere'] == 'Northern']
plt.figure(figsize=(10, 6))
plot_regression(northern_data['Latitude'], northern_data['Temperature'], 'Northern', 'Latitude', 'Temperature (°C)')
plt.show()


In [ ]:
southern_data = weather_data[weather_data['Hemisphere'] == 'Southern']
plt.figure(figsize=(10, 6))
plot_regression(southern_data['Latitude'], southern_data['Temperature'], 'Southern', 'Latitude', 'Temperature (°C)')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plot_regression(northern_data['Latitude'], northern_data['Humidity'], 'Northern', 'Latitude', 'Humidity (%)')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plot_regression(southern_data['Latitude'], southern_data['Humidity'], 'Southern', 'Latitude', 'Humidity (%)')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plot_regression(northern_data['Latitude'], northern_data['Cloudiness'], 'Northern', 'Latitude', 'Cloudiness (%)')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plot_regression(southern_data['Latitude'], southern_data['Cloudiness'], 'Southern', 'Latitude', 'Cloudiness (%)')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plot_regression(northern_data['Latitude'], northern_data['Wind Speed'], 'Northern', 'Latitude', 'Wind Speed (m/s)')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plot_regression(southern_data['Latitude'], southern_data['Wind Speed'], 'Southern', 'Latitude', 'Wind Speed (m/s)')
plt.show()
